In [ ]:
%scala
val tags = com.databricks.logging.AttributionContext.current.tags

//*******************************************
// GET VERSION OF APACHE SPARK
//*******************************************

// Get the version of spark
val Array(sparkMajorVersion, sparkMinorVersion, _) = spark.version.split("""\.""")

// Set the major and minor versions
spark.conf.set("com.databricks.training.spark.major-version", sparkMajorVersion)
spark.conf.set("com.databricks.training.spark.minor-version", sparkMinorVersion)

//*******************************************
// GET VERSION OF DATABRICKS RUNTIME
//*******************************************

// Get the version of the Databricks Runtime
val runtimeVersion = tags.collect({ case (t, v) if t.name == "sparkVersion" => v }).head
val runtimeVersions = runtimeVersion.split("""-""")
val (dbrVersion, scalaVersion) = if (runtimeVersions.size == 3) {
  val Array(dbrVersion, _, scalaVersion) = runtimeVersions
  (dbrVersion, scalaVersion.replace("scala", ""))
} else {
  val Array(dbrVersion, scalaVersion) = runtimeVersions
  (dbrVersion, scalaVersion.replace("scala", ""))
}
val Array(dbrMajorVersion, dbrMinorVersion, _) = dbrVersion.split("""\.""")

// Set the the major and minor versions
spark.conf.set("com.databricks.training.dbr.major-version", dbrMajorVersion)
spark.conf.set("com.databricks.training.dbr.minor-version", dbrMinorVersion)

//*******************************************
// GET USERNAME AND USERHOME
//*******************************************

// Get the user's name
val username = tags.getOrElse(com.databricks.logging.BaseTagDefinitions.TAG_USER, java.util.UUID.randomUUID.toString.replace("-", ""))

// Get the user's home. Create it if necessary
val userhome = s"dbfs:/user/$username"
// This call doesn't fail if it already exists
val created = dbutils.fs.mkdirs(userhome)

// Set the user's name and home directory
spark.conf.set("com.databricks.training.username", username)
spark.conf.set("com.databricks.training.userhome", userhome)

//**********************************
// VARIOUS UTILITY FUNCTIONS
//**********************************

def assertSparkVersion(expMajor:Int, expMinor:Int):String = {
  val major = spark.conf.get("com.databricks.training.spark.major-version")
  val minor = spark.conf.get("com.databricks.training.spark.minor-version")

  if ((major.toInt < expMajor) || (major.toInt == expMajor && minor.toInt < expMinor))
    throw new IllegalArgumentException(s"This notebook must be ran on Spark version $expMajor.$expMinor or better, found Spark $major.$minor")

  s"$major.$minor"
}

def requireDbrVersion(version:String):Unit = {
  val major = spark.conf.get("com.databricks.training.dbr.major-version")
  val minor = spark.conf.get("com.databricks.training.dbr.minor-version")
  val dbrVersion = major+"."+minor
  
  if (version != dbrVersion) {
    throw new IllegalArgumentException(s"This notebook must be ran on Databricks Runtime (DBR) $version version, found DBR $dbrVersion.")
  }
}

def assertDbrVersion(expMajor:Int, expMinor:Int):String = {
  val major = spark.conf.get("com.databricks.training.dbr.major-version")
  val minor = spark.conf.get("com.databricks.training.dbr.minor-version")

  if ((major.toInt < expMajor) || (major.toInt == expMajor && minor.toInt < expMinor))
    throw new IllegalArgumentException(s"This notebook must be ran on Databricks Runtime (DBR) version $expMajor.$expMinor or better, found DBR $major.$minor.")
  
  s"$major.$minor"
}

displayHTML(s"""
  <div>Successfully created class variables and utility functions.</div>
  <li><b>username</b>: $username</li>
  <li><b>userhome</b>: $userhome</li>
""")

In [ ]:
%python

from __future__ import print_function
from __future__ import division

#**********************************
# VARIOUS UTILITY FUNCTIONS
#**********************************

def assertSparkVersion(expMajor, expMinor):
  major = spark.conf.get("com.databricks.training.spark.major-version")
  minor = spark.conf.get("com.databricks.training.spark.minor-version")

  if (int(major) < expMajor) or (int(major) == expMajor and int(minor) < expMinor):
    msg = "This notebook must run on Spark version {}.{} or better, found.".format(expMajor, expMinor, major, minor)
    raise Exception(msg)

  return major+"."+minor

def assertDbrVersion(expMajor, expMinor):
  major = spark.conf.get("com.databricks.training.dbr.major-version")
  minor = spark.conf.get("com.databricks.training.dbr.minor-version")

  if (int(major) < expMajor) or (int(major) == expMajor and int(minor) < expMinor):
    msg = "This notebook must run on Databricks Runtime (DBR) version {}.{} or better, found.".format(expMajor, expMinor, major, minor)
    raise Exception(msg)
    
  return major+"."+minor

#**********************************
# INIT VARIOUS VARIABLES
#**********************************

username = spark.conf.get("com.databricks.training.username")
userhome = spark.conf.get("com.databricks.training.userhome")

import sys
pythonVersion = spark.conf.set("com.databricks.training.python-version", sys.version[0:sys.version.index(" ")])

None # suppress output